In [3]:
import os
import sys
initial_path = set(sys.path)
CARLA_EGG_PATH = "/home/rudy/Documents/PythonAPI/carla/dist/carla-0.9.11-py3.7-linux-x86_64.egg"
sys.path.append(CARLA_EGG_PATH)

# ADD 
try:
    sys.path.append(os.path.abspath('.') + '/PythonAPI/carla')
except IndexError:
    pass

new_paths = set(sys.path) - initial_path
for path in new_paths:
    print(f"Added: {path} to the Path")

import carla

In [179]:
import random
import math

In [325]:
client = carla.Client('localhost', 2000)
client.set_timeout(20.0)
client.load_world('Town02')
world = client.get_world()
map_ = world.get_map()
blueprint_library = world.get_blueprint_library()
vehicle_bp = random.choice(blueprint_library.filter('vehicle.bmw.grandtourer'))

In [326]:
def draw_loc(l, label='x'):
    world.debug.draw_string(l, label, draw_shadow=False,
                        color=carla.Color(r=255, g=0, b=0), life_time=30.0,
                        persistent_lines=True)

In [330]:
t = map_.get_spawn_points()[0]
t.location.x += -0.0
t.rotation.yaw += 0

In [331]:
# spawn
actor = world.spawn_actor(vehicle_bp, t)

In [329]:
# destroy
actor.destroy()

False

In [1]:
# draw Box
transform = actor.get_transform()
bounding_box = actor.bounding_box
bounding_box.location += transform.location
world.debug.draw_box(bounding_box, transform.rotation)

NameError: name 'actor' is not defined

In [337]:
import numpy as np

def vec3d_dot(v1, v2):
    return v1.x * v2.x + v1.y * v2.y + v1.z * v2.z

def vec3d_mag(v):
    return np.sqrt(vec3d_dot(v, v))

def vec3d_angle(v1, v2, deg=False):
    cos_theta = vec3d_dot(v1, v2) / (vec3d_mag(v1) * vec3d_mag(v2))
    if deg:
        return np.rad2deg(np.arccos(cos_theta))
    return np.arccos(cos_theta)

def get_vehicle_orientation(world_map, vehicle):
    """
    Returns the vehicle's orientation w.r.t to lane's tangent
    """
    
    # get lane tangent vector
    a1 = world_map.get_waypoint(vehicle.get_location(), project_to_road=True)
    b1 = a1.next(2.0)[0]
    vector_lane = b1.transform.location - a1.transform.location
    
    # get vehicle vector
    vertices = vehicle.bounding_box.get_world_vertices(vehicle.get_transform())
    a2 = vertices[0]
    b2 = vertices[4]
    vector_vehicle = b2 - a2
    
    return vec3d_angle(vector_vehicle, vector_lane, deg=True)

def get_vehicle_position(world_map, vehicle):
    """
    Returns the distance of the vehicle to lane's center
    """
    vehicle_center = vehicle.get_transform()
    lane_center = world_map.get_waypoint(vehicle.get_location(), project_to_road=True)
    d_x = vehicle_center.location.x - lane_center.transform.location.x
    d_y = vehicle_center.location.y - lane_center.transform.location.y
    d = np.sqrt(d_x * d_x + d_y * d_y)
    return d
    
    

In [338]:
print(get_vehicle_position(map_, actor))
print(get_vehicle_orientation(map_, actor))

0.11462126397980087
0.1288645516281961
